In [2]:
import numpy as np
import torch
from torch.distributions.multivariate_normal import MultivariateNormal

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from skimage import io
import os


from numba import cuda
import math
import requests

from queue import PriorityQueue

In [3]:
plotly.__version__
pio.renderers.default = "iframe"

In [4]:
np.__version__

'1.24.3'

In [5]:
np.random.seed(0)

In [6]:
available_gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]
available_gpus

In [7]:
%load_ext autoreload
%autoreload 2

from model import Bicycle, Map
from mppi import MPPI, MPPI_path_follower
from path_planner import a_star_planner
from sim import Simulation

import kmeans

In [8]:
torch.cuda.empty_cache() 

In [9]:
# avoid = []

# rect = [(20, 0, 80, 50),(20,51,80, 100) ]
# circle= []

# state_dim = 4
# ctrl_dim = 2
# u_min = (0., -3.)
# u_max = (10., 3.)
# step = .1
# follow_depth = 16 # note: if these parameters are changed, the mppi ones should be changed to match
# horizon_depth = 32

# start at 1-5, 45-55, angle -0

In [10]:

# avoid = []

# rect = [(0, 0, 100, 10),(0, 0, 10, 100), (0, 61, 85, 100), (20, 0, 100, 59),(95,0,100,100) ]
# circle= [(20,40, 6), (10, 30, 6),(10, 50, 6)]

# state_dim = 4
# ctrl_dim = 2
# u_min = (0., -3.)
# u_max = (10., 3.)
# step = .1
# follow_depth = 16 # note: if these parameters are changed, the mppi ones should be changed to match
# horizon_depth = 32

#start at (11-19, 11-19), angle math.pi/2

In [11]:
avoid = []

rect = [(0, 15, 47, 85), (53, 15, 100,85)]
left_circles = [(47,20+x, 5) for x in range(0, 80, 20)]
print(left_circles)
right_circles = [(53,30+x, 5) for x in range(0, 60, 20)]
circle= left_circles + right_circles

state_dim = 4
ctrl_dim = 2
u_min = (0., -3.)
u_max = (10., 3.)
step = .1
follow_depth = 16 # note: if these parameters are changed, the mppi ones should be changed to match
horizon_depth = 32

#start at (45-55, 1-5), angle math.pi/2

[(47, 20, 5), (47, 40, 5), (47, 60, 5), (47, 80, 5)]


In [12]:
def get_expert_controls(control_history, horizon_depth, ctrl_dim, interval_start = 0, interval_end = None, interval_step=2, dev="cpu"):
    if(interval_end is None):
        interval_end = len(control_history)-1
    first_rollout = interval_start
    last_rollout = interval_end-horizon_depth
    
    rollout_starts = range(first_rollout, last_rollout+1, interval_step)
    size = len(rollout_starts)
    
    expert_controls = torch.empty([size, horizon_depth, ctrl_dim], dtype = torch.float, device = dev)
    count = 0
    for i in rollout_starts:
        expert_controls[count,:,:] = torch.stack(control_history[i:i+horizon_depth])        
        count+=1
    return expert_controls

In [13]:
available_gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]

In [14]:
expert_controls = torch.tensor([],device = torch.device('cuda:0')) # used to reset expert_controls from constantly appending
a_star_sim = None
#
for i in range(45, 56, 2):
    for j in [1, 5]:
        print(i, j)
        a_star_robot = Bicycle(i, j, math.pi/2, 0)
        goal = (50, 90, 10 , 100)

        mppi_sample_center= torch.tensor([0., 0.])
        mppi_sample_covariance = torch.tensor([[16.,0.],[0., 4.]])

        env = Map(goal_point = goal, avoidance_points=avoid, rect_obstacles=rect,circle_obstacles=circle, device = torch.device('cuda:0'))
        a_star_map = Map(goal_point = (a_star_robot.x, a_star_robot.y,2,10), speed_weight = 1, avoidance_points=avoid, 
                         rect_obstacles=rect,circle_obstacles=circle, device = torch.device('cuda:0'))

        def a_star_heuristic(state):     
            return env.terminal_state_cost_batch(state)

        a_star_plan = a_star_planner(a_star_robot, state_dim, ctrl_dim, u_min, u_max, 
                          env.get_obstacles_batch, env.get_distance_batch,
                         iterations = 50000, goal_point = env.goal_point, angle_density = 32, grid_interval = 0.2, goal_tolerance =0.1, 
                                           timestep = step, device=torch.device('cuda:0'))
        
        print ("plan ready")
        a_star_follower = MPPI_path_follower(a_star_robot, state_dim, ctrl_dim, 
                          mppi_sample_center, mppi_sample_covariance,  u_min, u_max, 
                          a_star_robot.dynamics_batch_horizon, a_star_map, 
                                             a_star_map.running_cost_batch_horizon, a_star_map.terminal_state_cost_batch, a_star_plan.path, 
                         num_samples = 4096, horizon = follow_depth, lambda_=.1, sample_null_action = True,
                          timestep = step, device=torch.device('cuda:0'))
        print("expert demo started")
        
        a_star_sim = Simulation(a_star_robot, a_star_follower, a_star_map, goal_tolerance = 1, timestep = step)
        
        a_star_sim.run(200, False, False, False, 10, 1)
        
        a_star_sim.display_history()
        

        new_controls = get_expert_controls(a_star_robot.control_history, horizon_depth, ctrl_dim,interval_step = 2, dev = torch.device('cuda:0'))
        
        expert_means, assigned_num, loss = kmeans.k_means_segment(new_controls, k = 7)
        expert_controls = torch.cat((expert_controls, expert_means), 0)

        expert_means[:, :, -1] = -expert_means[:, :, -1]
        expert_controls = torch.cat((expert_controls, expert_means), 0)

        
        # losses = []
        # clusters = []
        
        # for num_cluster in range(1, 20, 2):
        #     expert_means, assigned_num, loss = kmeans.k_means_segment(new_controls, k = num_cluster)
        #     losses.append(loss)
        #     clusters.append(num_cluster)
        # plt.plot(clusters, losses)
        # plt.show()
        
        print(expert_controls.shape)



45 1
plan ready
expert demo started


torch.Size([14, 32, 2])
45 5
plan ready
expert demo started


torch.Size([28, 32, 2])
47 1
plan ready
expert demo started


torch.Size([42, 32, 2])
47 5
plan ready
expert demo started


torch.Size([56, 32, 2])
49 1
plan ready
expert demo started


...

torch.Size([112, 32, 2])
53 1
plan ready
expert demo started


torch.Size([126, 32, 2])
53 5
plan ready
expert demo started


torch.Size([140, 32, 2])
55 1
plan ready
expert demo started


torch.Size([154, 32, 2])
55 5
plan ready
expert demo started


torch.Size([168, 32, 2])


In [24]:
'''
a_star_sim.draw_explored(a_star_plan.path, False)

a_star_sim.display_run()
a_star_sim.display_history()
'''

'\na_star_sim.draw_explored(a_star_plan.path, False)\n\na_star_sim.display_run()\na_star_sim.display_history()\n'

In [25]:
# num_clusters = 12
# expert_means, assigned_to_center, loss = kmeans.k_means_segment(expert_controls, k = num_clusters)
# print(assigned_to_center, expert_means.shape)

In [24]:
use_expert_controls = True
samples = 2048
controls = None


expert_samples = [5 for i in range(168)]

if use_expert_controls:
    if expert_samples is not None:
        #samples = samples - sum(expert_samples)
        controls = expert_controls
    else:
        samples = samples - expert_controls.shape[0]
        controls = expert_controls
    
if(samples <0):
    print("too many expert rollouts")

In [28]:

mppi_sample_center= torch.tensor([0., 0.])
mppi_sample_covariance = torch.tensor([[16.,0.],[0., 1]])
# note: if these parameters are changed, the a-star ones should be changed to match

# changes to 8 clusters

success = 0.
total = 0.
total_it = 0.

for i in [45, 50, 55]:
    for j in [1,  5]:
        mppi_robot = Bicycle(i, j, math.pi/2, 0)
        mppi_goal = (50, 90, 3, 10)

        mppi_map = Map(goal_point = mppi_goal, avoidance_points=avoid, 
                       rect_obstacles=rect,circle_obstacles=circle, speed_weight = 1, device = torch.device('cuda:0'))
        mppi_controller = MPPI(mppi_robot, state_dim, ctrl_dim, 
                          mppi_sample_center, mppi_sample_covariance,  u_min, u_max,
                          mppi_robot.dynamics_batch_horizon, mppi_map.running_cost_batch_horizon, mppi_map.terminal_state_cost_batch,
                               expert_rollouts = controls, expert_samples = expert_samples,
                          num_samples = samples, horizon = horizon_depth, lambda_=.1,sample_null_action = True,
                          timestep = step, device=torch.device('cuda:0'))
        mppi_sim = Simulation(mppi_robot, mppi_controller, mppi_map, timestep = step, goal_tolerance = 2)

        iterations, completed = mppi_sim.run(iterations=200, draw_obstacles = False, write_snapshots =True, 
              write_rollouts=True, write_controls = True, write_rollout_start = 0, write_rollouts_num = 5,write_rate = 2)
        mppi_sim.display_run()
        mppi_sim.display_history()
        print(i, j, completed, iterations)

        if completed:
            success+=1
            total_it += iterations
        total+=1
        
print(success/total)
if success > 0:
    print(total_it/success)



45 1 False 200


45 5 True 178


50 1 True 145


50 5 False 200


55 1 False 200


55 5 True 147
0.5
156.66666666666666
